In [ ]:
import pandas as pd
import numpy as np
from pybnesian import hc, CLGNetworkType, SemiparametricBNType
#from drawdata import draw_scatter
import matplotlib.pyplot as plt
import math

from pymoo.core.problem import ElementwiseProblem

from pymoo.optimize import minimize

from bayesace.algorithms.face import face_algorithm
from bayesace.utils import *
from bayesace.algorithms.bayesace import BayesACE
from bayesace.algorithms.face import FACE

from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_csv("toy-3class.csv")
df["class"] = df["z"].astype('category')
df = df.drop("z", axis = 1 )

feature_columns = [i for i in df.columns if i != "class"]
df[feature_columns] = StandardScaler().fit_transform(df[feature_columns].values)



learned = hc(df, bn_type = CLGNetworkType(), operators = ["arcs"], score = "validated-lik")
learned.fit(df)

learned_kde = hc(df, bn_type = SemiparametricBNType(), operators = ["arcs", "node_type"], score = "validated-lik")
learned_kde.fit(df)



In [ ]:
predict_class(df, learned_kde)

In [ ]:
for i in learned.nodes() :
    print(i)
    print(learned.cpd(i))

In [ ]:
instance = df.iloc[[0]]
instance["class"] = pd.Categorical(["False"], categories=learned.cpd("class").variable_values())
math.e ** learned.logl(instance)

In [ ]:
n_vertex = 0
x_og = df.iloc[[0]]
bayesace = BayesACE(bayesian_network=learned_kde, features= df.columns[:-1], n_vertex=n_vertex, chunks = 100, penalty=1, pop_size=50, likelihood_threshold=0.0, accuracy_threshold= 0.5, generations=10)
res = bayesace.run(x_og)

In [ ]:
res.counterfactual

In [ ]:
df.iloc[0]

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.show()

In [ ]:
n_vertex = 1
x_og = df.iloc[[0]]
face = FACE(bayesian_network=learned_kde, features= df.columns[:-1], dataset=df.drop("class", axis = 1), graph_type = "epsilon", chunks = 10, penalty=1, distance_threshold=0.75, accuracy_threshold= 0.5, verbose = False)
res = face.run(x_og)

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.show()